In [55]:
import yaml
import collections
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go
from matplotlib.pyplot import get_cmap

cmap=get_cmap("tab10")

In [56]:
with open("../../../results/20240131-test/diarization.yaml", "r") as f:
    diarization = yaml.safe_load(f)

speakers = []
for item in diarization:
    speakers.append(item["speaker"])

In [57]:
with open("../../../results/20240131-test/result.yaml", "r") as f:
    conversation = yaml.safe_load(f)
# conversation

In [58]:
timings = np.arange(int(conversation["segments"][-1]["end"]))
# timings

In [59]:
def who_speak(t, diar=diarization):
    for item in diar:
        if item["start"] < t and t < item["end"]:
            return item["speaker"]
    return "none"

In [60]:
max_delta = 4
_labels = []
for _t in timings:
    for delta in range(max_delta):
        t = _t + delta/max_delta
        _labels.append([float(t), who_speak(t)])
labels = np.array(_labels)
# print(labels)

In [63]:
for segment in conversation["segments"]:
    _label_of_segment = []
    for t in np.arange(int(segment["start"]*max_delta)/max_delta, int(segment["end"]*max_delta)/max_delta, 0.25):
        idx = np.where(labels==str(t))[0].astype(np.int64)
        if len(labels[idx, -1]):
            _label_of_segment.append(labels[idx, -1])
    _labels_seg = np.array(_label_of_segment).flatten()
    labels_seg = _labels_seg[_labels_seg != "none"]
    counted = collections.Counter(labels_seg)
    segment_label = counted.most_common()[0][0]
    segment["label"] = str(segment_label)

In [66]:
# conversation

In [65]:
with open("../../../results/20240131-test/diarized-result.yaml", "w") as f:
    yaml.dump(conversation, f, default_flow_style=False, indent=4, allow_unicode=True, sort_keys=False)